In [ ]:
import os
os.environ['ISISROOT'] = '/usgs/cpkgs/anaconda3_linux/envs/isis3.8.1'
os.environ['PATH'] += ':/home/jlaura/anaconda3/envs/krc/bin/'
import krc

%pylab inline

In [ ]:
params = {"albedo": [0.08,0.22,0.32],
          "slope_azimuth": [0],
          "slope": [0.0],
          "tau": [0.02,0.30,0.62],
          "elevation": [-5.0,-2.0,-1.0,1.0,6.0,8.0],
          "inertia": [40, 250, 400, 800, 1200],
          "emissivity": [1.00]
         }

header = """0 0 / KOLD: season to start with;  KEEP: continue saving data in same disk file
Version {} default values.  37 latitudes with mean Mars zonal elevations       
    ALBEDO     EMISS   INERTIA     COND2     DENS2    PERIOD SPEC_HEAT   DENSITY
       .25      1.00     200.0      2.77     928.0    1.0275      647.     1600.
      CABR       AMW    SatPrA    PTOTAL     FANON      TATM     TDEEP   SpHeat2
      0.11      43.5   27.9546     546.0      .055      200.     180.0     1711.
  TAUD/PHT     DUSTA    TAURAT     TWILI  ARC2/Pho ARC3/Safe     SLOPE    SLOAZI
       0.3       .94     0.204       0.0      0.65     0.801       0.0       90.
    TFROST    CFROST    AFROST     FEMIS       AF1       AF2    FROEXT    SatPrB
     146.0   589944.       .65      0.95      0.54    0.0009       50.   3182.48
      RLAY      FLAY     CONVF     DEPTH     DRSET       DDT       GGT     DTMAX
    1.1500     {}       3.0       0.0       0.0     .0020       0.1       0.1
      DJUL    DELJUL  SOLARDEC       DAU     LsubS    SOLCON      GRAV     AtmCp
  -1222.69   8.58713      00.0     1.465        .0     1368.     3.727     735.9
    ConUp0    ConUp1    ConUp2    ConUp3    ConLo0    ConLo1    ConLo2    ConLo3
  0.038640 -0.002145  0.002347 -0.000750  2.766722 -1.298966  0.629224 -0.527291
    SphUp0    SphUp1    SphUp2    SphUp3    SphLo0    SphLo1    SphLo2    SphLo3
  646.6275  246.6678  -49.8216    7.9520  1710.648  721.8740  57.44873  24.37532
        N1        N2        N3        N4        N5       N24       IIB       IC2
        38      1536        15        37       320        48         0         7
     NRSET      NMHA      NRUN     JDISK     IDOWN    FlxP14 TUN/Flx15     KPREF
         3        24         0       241         0        45        65         1
     K4OUT     JBARE     Notif    IDISK2                                     end
        {}         0       200         0                                       0
    LP1    LP2    LP3    LP4    LP5    LP6 LPGLOB   LVFA   LVFT  LkofT          
      F      T      F      F      F      F      F      F      F      F          
  LPORB   LKEY    LSC  LZONE  LOCAL  Prt76 LPTAVE  Prt78  Prt79  L_ONE          
      T      F      F      F      T      F      F      F      F      F          
Latitudes: in 10F7.2  _____7 _____7 _____7 _____7 _____7 _____7 _____7          
 -90.00 -85.00 -80.00 -75.00 -70.00 -65.00 -60.00 -55.00 -50.00 -45.00          
 -40.00 -35.00 -30.00 -25.00 -20.00 -15.00 -10.00  -5.00   0.00   5.00          
  10.00  15.00  20.00  25.00  30.00  35.00  40.00  45.00  50.00  55.00          
  60.00  65.00  70.00  75.00  80.00  85.00  90.00                               
 _____7 _____7 _____7 Elevations: in 10F7.2 ____7 _____7 _____7 _____7          
{elev}                                                                              
 2013 Jul 24 11:28:09=RUNTIME.  IPLAN AND TC= 104.0 0.10000 Mars:Mars           
   104.0000      0.1000000      0.8644665      0.3226901E-01  -1.281586         
  0.9340198E-01   1.523712      0.4090926       0.000000      0.9229373         
   5.544402       0.000000       0.000000       686.9929       3397.977         
   24.62296       0.000000      -1.240317       0.000000       0.000000         
   0.000000      0.3244965      0.8559126      0.4026359     -0.9458869         
  0.2936298      0.1381285       0.000000     -0.4256703      0.9048783"""

This block generates the change cards for the two different model runs. For the current setup, it takes about 2:30 to create the `.tds` files for  

In [ ]:
# Variables to set

# Name of the directories where the test run will be written
testdir_354 = '/scratch/jlaura/krc_compare/t2_354'
testdir_362 = '/scratch/jlaura/krc_compare/t2_362'

In [ ]:
from krc.changecards import changecards

def get_header(version=354):
    """
    This create a change card header for version >= 3.5.4.
    """
    return header.format(354, .115, -2, elev={}) # This is setting the version number, flay, and k4out
def get_321_header():
    return header.format(321, .100, -1, elev={})

# These are the names of the change cards
cname_354 = os.path.join(testdir_354, 'o_354')
cname_352 = os.path.join(testdir_362, 'o_362')

# Create the change cards
changecards.createchangecards(get_header(version=354), params,
                              cname_354,
                              testdir_354,
                              log_inertias=False)
changecards.createchangecards(get_header(version=362), params,
                              cname_362,
                              testdir_362,
                              log_inertias=False)

#Copy the 343 and 321 binaries into the respective directories.
krc_354 = os.path.join(testdir_354, 'krc')
krc_321 = os.path.join(testdir_362, 'krc')

# And Copy Robin's compiled versions
shutil.copyfile('/home/rfergason/bin/Krc/astrovm4/krc_354_2017Oct/run/krc', krc_354)
#shutil.copyfile('/home/rfergason/bin/Krc/astrovm4/krc_321_2015MarchChanges/run/krcd', krc_321)
shutil.copyfile('/home/rfergason/bin/Krc/astrovm4/krc_362_2019March/src/krc', krc_362)
os.chmod(krc_354, 755)
os.chmod(krc_362, 755)

submitjob(testdir_362, '.inp', krc_362, 362, testdir_362)
submitjob(testdir_354, '.inp', krc_354, 354, testdir_354)

# Watch the queue

I like to use `watch -n 2 squeue` on Astrovm4 to see how the queue is doing.  The above cell should generate 270 files in two directories (in the above case, `t1_321` and `t1_354`). I also like to use `ls *.tds | wc -l` to check to on V354 to see if I need to rerun the above cell. Robin, you need to have 270 `.tds` files. 

In [ ]:
seasonal_354 = changecards.getfiles(testdir_354, params, 354, log_inertias=False)
seasonal_321 = changecards.getfiles(testdir_321, params, 321, log_inertias=False)

## Interpreting the plots

The first (dashed seperators) set of parameters are the thermal inertia.  Within each thermal inertia, groups of 5 are the six different elevation parameters.  Within each elevation are 3 opacities and finally 3 albedos.

In [ ]:
%pylab inline
figsize(12,12)
nparams = 270
step = int(nparams/5)

def descriptive_grid(ax, arr, title):
    ax.plot(np.arange(nparams),arr, 'bo', alpha=0.25, markersize=3)
    for j in range(step,nparams,step):
        ax.plot((j, j), (np.min(arr)-1,np.max(arr)+1), 'k--', alpha=0.5)
    for k in range(9,270,9):
        if k % 54 == 0:
            continue
        ax.plot((k+.5, k+.5), (np.min(arr)-1,np.max(arr)+1), 'r--', alpha=0.15, linewidth=1)
    ax.title.set_text(title)
    ax.set_xticks([27,81,135,189,243])
    ax.set_xticklabels([40, 250, 400, 800, 1200])
    ax.tick_params('y', colors='k')
    ax.set_xlabel('sk')
    ax.set_ylabel('K')
    return ax

for i in range(80):
    print('Season {}'.format(i))
    a = seasonal_354[i]
    b = seasonal_321[i]
    fig,axes = plt.subplots(2,2)
    # First index is time
    # Second index is lat
    # Third index is the parameter space
    lat_indices = np.arange(-90, 95, 5)
    time_indices = np.arange(0,48.5, 0.5)
    no_poles = np.where((lat_indices <= 45) & (lat_indices >= -45))[0]
    a = a[:,no_poles,:]
    b = b[:,no_poles,:]
    # Difference the two arrays
    diff = b-a
    
    # Plot the min difference
    mindiff = np.min(diff, axis=(0,1))
    descriptive_grid(axes[0][0], mindiff, 'min')

    # Plot the maximum difference
    maxdiff = np.max(diff, axis=(0,1))
    descriptive_grid(axes[0][1], maxdiff, 'max')

    # Plot the mean difference
    meandiff = np.mean(diff, axis=(0,1))
    descriptive_grid(axes[1][0], meandiff, 'mean')

    # Plot the std difference
    stddiff = np.std(diff, axis=(0,1))
    descriptive_grid(axes[1][1], stddiff, 'std')
    show()
    print()